From : [링크](https://dacon.io/competitions/official/235747/codeshare/3047?page=2&dtype=recent)

In [1]:
import os
import warnings

import numpy as np
import pandas as pd
import datasets
from datasets import load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import EarlyStoppingCallback

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader

import wandb

# warnings.filterwarnings(action='ignore')

C:\Users\Kyeul\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


klue/bert-base 모델

In [2]:
wandb.init(
    project='boong_u__klue-ynat_classification',
    entity='gyul611',
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gyul611. Use `wandb login --relogin` to force relogin


In [3]:
model_checkpoint = 'klue/bert-base'
batch_size=512+256
task='nli' # 자연어 추론
RANDOM_SEED=42

In [4]:
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [5]:
dataset = pd.read_csv("../data/train_data.csv")
test = pd.read_csv("../data/test_data.csv")

In [6]:
dataset_train, dataset_val = train_test_split(dataset, test_size=0.2, random_state=RANDOM_SEED)

In [7]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_key, label_key, bert_tokenizer):
        self. sentences = [bert_tokenizer(i, truncation=True, return_token_type_ids=False) for i in dataset[sent_key]]

        # ? 왜 이렇게 하는 거지

        # label이 있으면 train 모드
        # 없으면 test 모드
        if not label_key == None:
            self.mode = "train"
        else:
            self.mode = "test"

        # label 매핑?
        if self.mode == "train":
            self.labels = [np.int64(i) for i in dataset[label_key]]
        else:
            self.labels = [np.int64(0) for i in dataset[sent_key]]

    def __getitem__(self, i):
        if self.mode == "train":
            self.sentences[i]['label'] = self.labels[i]
            return self.sentences[i]

        else:
            return self.sentences[i]

    def __len__(self):
        return (len(self.labels))

In [8]:
data_train = BERTDataset(dataset_train, "title", "topic_idx", tokenizer)
data_val = BERTDataset(dataset_val, "title", "topic_idx", tokenizer)
data_test = BERTDataset(test, "title", None, tokenizer)

In [9]:
num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
metric = load_metric("glue", "qnli")

In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
metric_name = "accuracy"

args = TrainingArguments(
    'test-nli',
    save_total_limit=5,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    report_to='wandb',
)

In [13]:
# hyperparameter_search를 위해서는 model_init을 만들어야함.

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [14]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=data_train,
    eval_dataset=data_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
best_run = trainer.hyperparameter_search(n_trials=2, direction="maximize")

[I 2023-11-28 18:11:02,837] A new study created in memory with name: no-name-229d4ca8-f7cb-47ba-9f08-1137b59f99dd
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.602700,0.346917,0.884898
2,0.332700,0.322956,0.891907
3,0.295100,0.320853,0.893330
4,0.270100,0.317120,0.893988
5,0.242500,0.316513,0.896178


[I 2023-11-28 18:15:04,399] Trial 0 finished with value: 0.8961778556565546 and parameters: {'learning_rate': 6.931763037713002e-06, 'num_train_epochs': 5, 'seed': 23, 'per_device_train_batch_size': 64}. Best is trial 0 with value: 0.8961778556565546.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/accuracy,▁▅▆▇█
eval/loss,█▂▂▁▁
eval/runtime,▆█▁▂▂
eval/samples_per_second,▃▁█▇▇
eval/steps_per_second,▃▁█▇▇
train/epoch,▁▁▂▃▄▅▅▆▇██
train/global_step,▁▁▂▃▄▅▅▆▇██
train/learning_rate,█▆▅▃▁
train/loss,█▃▂▂▁
train/total_flos,▁
train/train_loss,▁


Epoch,Training Loss,Validation Loss,Accuracy
1,0.371200,0.328187,0.890483
2,0.300000,0.317756,0.893002
3,0.249400,0.321151,0.895849
4,0.220000,0.322583,0.897821
5,0.188600,0.331204,0.897711


[I 2023-11-28 18:20:33,047] Trial 1 finished with value: 0.8977110940751287 and parameters: {'learning_rate': 8.17142456246069e-06, 'num_train_epochs': 5, 'seed': 35, 'per_device_train_batch_size': 32}. Best is trial 1 with value: 0.8977110940751287.


In [16]:
best_run

BestRun(run_id='1', objective=0.8977110940751287, hyperparameters={'learning_rate': 8.17142456246069e-06, 'num_train_epochs': 5, 'seed': 35, 'per_device_train_batch_size': 32}, run_summary=None)

In [17]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.371200,0.328187,0.890483
2,0.300000,0.317756,0.893002
3,0.249400,0.321151,0.895849
4,0.220000,0.322583,0.897821
5,0.188600,0.331204,0.897711


TrainOutput(global_step=5710, training_loss=0.2858988927249942, metrics={'train_runtime': 316.8462, 'train_samples_per_second': 576.352, 'train_steps_per_second': 18.021, 'total_flos': 2056542634442700.0, 'train_loss': 0.2858988927249942, 'epoch': 5.0})

In [18]:
trainer.evaluate() # 

{'eval_loss': 0.3225833475589752,
 'eval_accuracy': 0.8978206111050269,
 'eval_runtime': 2.9511,
 'eval_samples_per_second': 3094.069,
 'eval_steps_per_second': 4.066,
 'epoch': 5.0}

In [19]:
pred = trainer.predict(data_test)
pred

PredictionOutput(predictions=array([[ 1.7568895 ,  0.1507784 ,  2.1030583 , ..., -2.3661284 ,
        -2.6856828 , -3.3943496 ],
       [-1.8309041 , -2.0773034 ,  0.10060618, ..., -1.166829  ,
        -1.552108  , -1.3320658 ],
       [ 2.033948  ,  0.40936077,  4.301589  , ..., -2.6633341 ,
        -2.5798059 , -0.5348093 ],
       ...,
       [-1.5713437 , -1.4163268 ,  3.2175384 , ..., -1.4705619 ,
        -2.3041637 , -2.1479445 ],
       [ 1.2939097 , -0.10006521,  4.077904  , ..., -2.964298  ,
        -1.5077595 , -0.8723607 ],
       [ 2.113509  , -0.9350316 ,  2.9323263 , ..., -2.2945492 ,
        -2.0192602 ,  1.9491215 ]], dtype=float32), label_ids=None, metrics={'test_runtime': 2.9082, 'test_samples_per_second': 3139.737, 'test_steps_per_second': 4.126})

In [20]:
pred = pred[0]
pred = np.argmax(pred, axis = 1)

In [22]:
submission = pd.read_csv("../data/sample_submission.csv")
submission['topic_idx'] = pred
submission.to_csv("klue-bert-base.csv", index=False)